### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [10]:
# from dataclasses import dataclass
# from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
# from autogen_agentchat.agents import AssistantAgent
# from autogen_agentchat.messages import TextMessage
# from autogen_ext.models.openai import OpenAIChatCompletionClient
# from autogen_ext.tools.langchain import LangChainToolAdapter
# from langchain_community.utilities import GoogleSerperAPIWrapper
# from langchain.agents import Tool
# from IPython.display import display, Markdown

from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

# from dotenv import load_dotenv

# load_dotenv(override=True)

# ALL_IN_ONE_WORKER = False

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = True

### Start with our Message class

In [2]:

# @dataclass
# class Message:
#     content: str

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [5]:
# from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

# host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
# host.start() 

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start()

### Let's reintroduce a tool

In [6]:
# serper = GoogleSerperAPIWrapper()
# langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
# autogen_serper = LangChainToolAdapter(langchain_serper)

serper = GoogleSerperAPIWrapper()
langchain_serper = Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [7]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [9]:
# class Player1Agent(RoutedAgent):
#     def __init__(self, name: str) -> None:
#         super().__init__(name)
#         model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
#         self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

#     @message_handler
#     async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
#         text_message = TextMessage(content=message.content, source="user")
#         response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
#         return Message(content=response.chat_message.content)

class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
# class Player2Agent(RoutedAgent):
#     def __init__(self, name: str) -> None:
#         super().__init__(name)
#         model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
#         self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

#     @message_handler
#     async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
#         text_message = TextMessage(content=message.content, source="user")
#         response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
#         return Message(content=response.chat_message.content)

class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
 
    
# class Judge(RoutedAgent):
#     def __init__(self, name: str) -> None:
#         super().__init__(name)
#         model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
#         self._delegate = AssistantAgent(name, model_client=model_client)
        
#     @message_handler
#     async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
#         message1 = Message(content=instruction1)
#         message2 = Message(content=instruction2)
#         inner_1 = AgentId("player1", "default")
#         inner_2 = AgentId("player2", "default")
#         response1 = await self.send_message(message1, inner_1)
#         response2 = await self.send_message(message2, inner_2)
#         result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
#         judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
#         message = TextMessage(content=judgement, source="user")
#         response = await self._delegate.on_messages([message], ctx.cancellation_token)
#         return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)

class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"##Pros of AutoGen:\n{response1.content}\n\n##Cons of AutoGen\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [11]:
# from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

# if ALL_IN_ONE_WORKER:

#     worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
#     await worker.start()

#     await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
#     await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
#     await Judge.register(worker, "judge", lambda: Judge("judge"))

#     agent_id = AgentId("judge", "default")

# else:

#     worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
#     await worker1.start()
#     await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

#     worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
#     await worker2.start()
#     await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

#     worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
#     await worker.start()
#     await Judge.register(worker, "judge", lambda: Judge("judge"))
#     agent_id = AgentId("judge", "default")

if ALL_IN_ONE_WORKER:
    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId('judge', 'default')

else:
    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")


In [12]:
# response = await worker.send_message(Message(content="Go!"), agent_id)
response = await worker.send_message(Message(content="Go!"), agent_id)

In [13]:
# display(Markdown(response.content))
display(Markdown(response.content))

##Pros of AutoGen:
Here are several reasons in favor of choosing AutoGen for your AI agent project:

1. **Advanced AI Capabilities**: AutoGen is designed for complex automation and can effectively handle dynamic environments, making it suitable for sophisticated applications.

2. **Multi-agent Collaboration**: It excels at fostering collaboration among multiple AI agents, which can enhance problem-solving capabilities through brainstorming, critique, and task completion.

3. **Customization**: AutoGen offers high levels of customization, allowing developers to tailor the framework to meet specific project needs and requirements.

4. **Facilitates Complex Interactions**: The framework supports back-and-forth interactions between AI roles, which is beneficial for projects involving decision-making and research.

5. **Rapid Development**: Its open-source nature combined with user-friendly design allows for quicker development cycles and iterative improvements.

6. **Integration with LLMs**: AutoGen effectively optimizes large language models (LLMs), providing robust support for building powerful language-based AI applications.

7. **Human-AI Interaction**: It incorporates strategies for human involvement during agent conversations, enhancing the interactive experience and ensuring that human insights are integrated into decision-making processes.

These advantages can significantly enhance the effectiveness and efficiency of your AI agent project.

##Cons of AutoGen
Here are some cons of using AutoGen for your AI Agent project:

1. **Steep Learning Curve**: Many users find that the initial learning curve is quite steep, especially for those unfamiliar with state machines or graph-based logic. This can require significant investment in time to become proficient.

2. **Documentation Challenges**: The documentation is reported to be hard to read and lacks sufficient examples, which can hinder the onboarding process and make troubleshooting difficult for new users.

3. **Cost Considerations**: AutoGen often involves usage-based pricing that can become expensive, particularly if workflows are not efficient. There is no pay-as-you-go option, which may necessitate an upgrade even if the usage does not justify it.

4. **Creativity Limitations**: While it is effective in generating content based on predefined templates, AutoGen may lack flexibility in creative scenarios, limiting its applicability in projects that require more innovative outputs.

5. **Complexity of Integration**: Depending on the existing systems and libraries used in your project, integrating AutoGen might introduce unnecessary complexity, especially if the team is already accustomed to other frameworks.

These points can help you weigh the considerations against using AutoGen for your AI Agent project. 

TERMINATE



## Decision:

Based on the pros and cons presented by the research team, I recommend using AutoGen for the project. The advanced AI capabilities, strong multi-agent collaboration features, and customization options align well with the project's goals, especially for complex automations and dynamic environments. Despite concerns around the steep learning curve and documentation challenges, the potential for rapid development and effective integration with large language models outweighs the drawbacks. Additionally, the benefits of enhanced human-AI interaction and iterative improvements are crucial for ensuring the project's success.

TERMINATE

In [14]:
# await worker.stop()
# if not ALL_IN_ONE_WORKER:
#     await worker1.stop()
#     await worker2.stop()

await worker.stop()

if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [15]:
# await host.stop()

await host.stop()
